In [1]:
# Add all annotated brains to the viewer
import os, sys
import neuroglancer
import numpy as np
import pandas as pd
import ast
%load_ext autoreload
%autoreload 2

HOME = os.path.expanduser("~")
if os.path.exists(os.path.join(HOME, 'Projects/pipeline_utility')):
    DIR = os.path.join(HOME, 'Projects/pipeline_utility')
else:
    DIR = os.path.join(HOME, 'programming/pipeline_utility')

sys.path.append(DIR)
print(DIR)
from utilities.contour_utilities import get_contours_from_annotations, add_structure_to_neuroglancer, \
    create_full_volume, get_structure_colors
from utilities.imported_atlas_utilities import get_structures

/home/eddyod/programming/pipeline_utility
Connecting dklab@192.168.1.12:3306


In [2]:
xy_ng_resolution_um = 5
color_radius = 3
animal = 'MD589'
#x_length = 1000
#y_length = 750
z_length = 1500

#y_voxels = 1+int( y_length*0.46*(.46/xy_ng_resolution_um) + 0.5)
#x_voxels = 1+int( x_length*0.46*(.46/xy_ng_resolution_um) + 0.5)
#full_brain_volume_annotated = np.zeros((z_length,y_voxels,x_voxels), dtype=np.uint8)

In [4]:
CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
csvfile = os.path.join(CSV_PATH, f'{animal}_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))

hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))
structures = get_structures()
#
structures_arr = hand_annotations.name.unique()
annotation_structures = structures_arr.tolist()

In [11]:
structures = [a for a in annotation_structures if a in structures]

In [7]:
colors = get_structure_colors()

In [14]:
#full_brain_volume_annotated = np.zeros((z_length,1500,3000), dtype=np.uint8)
#print('Reset full brain volume shape:', full_brain_volume_annotated.shape)
for structure in structures:
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure)
        try:
            color = colors[sided]
        except:
            color = 100

    print(structure, color, end="\t")
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations, densify=4)
    if first_sec == 0 or last_sec == 0:
        print('No sections found')
        continue
    else:
        print('Section start, end:' ,first_sec, last_sec, end="\t")
        

    threshold = 1
    structure_volume, xyz_offsets = create_full_volume(contour_annotations, structure, first_sec, last_sec, \
        color_radius, xy_ng_resolution_um, threshold, color)
    
    x_start, y_start, z_start = xyz_offsets
    x_end = x_start + structure_volume.shape[2]
    y_end = y_start + structure_volume.shape[1]
    z_end = z_start + structure_volume.shape[0]
    print('X range', x_start, x_end, end="\t")
    print('Y range', y_start, y_end, end="\t")
    print('Z range', z_start, z_end)
    #full_brain_volume_annotated[z_start:z_end, y_start:y_end,x_start:x_end] += structure_volume

SC 18	Section start, end: 128 311	X range 1466 3952	Y range 410 1577	Z range 36 255
IC 11	Section start, end: 122 314	X range 1507 4109	Y range 415 1685	Z range 30 252
12N 2	Section start, end: 206 245	X range 2476 5423	Y range 1095 2638	Z range 114 267
AP 9	Section start, end: 215 230	X range 2644 5533	Y range 1085 2478	Z range 123 261
RtTg 17	Section start, end: 200 275	X range 1752 4060	Y range 1163 2720	Z range 108 291


In [ ]:
viewer = neuroglancer.Viewer()
viewer

In [ ]:
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['z', 'y', 'x'], units='um', scales=[20, 5, 5]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:

OUTPUT = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes', animal)
outfile = os.path.join(OUTPUT, 'full_brain_volume_annotated.npy')
print('full_brain_volume_annotated at', OUTPUT)
np.save(outfile, np.ascontiguousarray(full_brain_volume_annotated))